## Building a GPT

Companion notebook to the [Zero To Hero](https://karpathy.ai/zero-to-hero.html) video on GPT.

### Downloading Wikipedia dataset from Kaggle

In [ ]:
!pip install -q kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d mikeortman/wikipedia-sentences

 99% 312M/314M [00:10<00:00, 36.0MB/s]
100% 314M/314M [00:10<00:00, 32.0MB/s]


In [ ]:
!unzip '/content/wikipedia-sentences.zip' -d '/content'

Archive:  /content/wikipedia-sentences.zip
  inflating: /content/wikisent2.txt  


In [ ]:
import numpy as np
import random
import re

DATA_PATH = 'wikisent2.txt'

# load wikipedia sentences
with open(DATA_PATH, 'r') as f:
    lines = f.read().splitlines()

print('Total Lines:', len(lines))

Total Lines: 7871825


In [ ]:
# Selecting 250k lines from the dataset.
random.seed(42)
texts = random.choices(lines, k=250000)
del lines

In [ ]:
def preprocess(text):
    text = re.sub('@.*?\s+', '', text)  # Remove mentions
    text = re.sub('#.*?\s+', '', text)  # Remove hashtags
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text)  # Remove URLs
    text = re.sub(r'[^\w\s\'.]', '', text)  # Remove special characters except for single quotes and periods
    text = re.sub('\s+', ' ', text)  # Replace multiple spaces with a single space
    text = re.sub('^\d+\s*|^\d+\.\d+\s*|^\d+\.\d+\.\d+\s*', '', text)  # Remove digits at the start of sentences
    text = text.strip()  # Remove leading and trailing whitespace
    return text


In [ ]:
wiki_text = [preprocess(t) for t in texts]

### Select dataset

In [ ]:
# Select which dataset to be used for training GPT model
text = '\n'.join(wiki_text)

### Model Training

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0

torch.manual_seed(1337)


# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


0.209987 M parameters
step 0: train loss 4.3875, val loss 4.3896
step 100: train loss 2.6774, val loss 2.6653
step 200: train loss 2.5536, val loss 2.5541
step 300: train loss 2.4927, val loss 2.5000
step 400: train loss 2.4414, val loss 2.4388
step 500: train loss 2.3893, val loss 2.3936
step 600: train loss 2.3322, val loss 2.3375
step 700: train loss 2.2821, val loss 2.2870
step 800: train loss 2.2551, val loss 2.2483
step 900: train loss 2.2171, val loss 2.2173
step 1000: train loss 2.1899, val loss 2.2021
step 1100: train loss 2.1680, val loss 2.1555
step 1200: train loss 2.1539, val loss 2.1524
step 1300: train loss 2.1192, val loss 2.1215
step 1400: train loss 2.1009, val loss 2.1102
step 1500: train loss 2.0867, val loss 2.0982
step 1600: train loss 2.0817, val loss 2.0922
step 1700: train loss 2.0578, val loss 2.0799
step 1800: train loss 2.0483, val loss 2.0522
step 1900: train loss 2.0307, val loss 2.0284
step 2000: train loss 2.0123, val loss 2.0114
step 2100: train loss 1.

### Generation

In [ ]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


An companion theoriso pirted attralized on National Leed T Header Calcas the Ifter and specially beculation incture.
He muninal hendel succell tanisted wember a Gug greend of Osekly from Serbutk more whild ampres lungulary in 1940s and centernment to lowested with Leleased Hustral U.Breprent in Danoubagawail Raig Penly Soshas Mockalisberling justom couil a.
Mana arnown are 19 to the Cinning Boon.
It a delective sequed of it rage from the Pepresist was FLIE for bille on Odemic Commostods and cholod scollage.
Thy is is locations Aterns fere early endempions.
Hn the Counciture.
Warlon Louglor Brikhame Dicar teamment runce as Palip Mouns Lee and Ribber Tour.
Wheld and coned when as peaper lated in the secreenk and the stonalition operation bivilled ots Connal PPI in a Gelds Kenam and UNWCEU is a freed hele as entwopled in elignes.
It inguites teapeal in EPniciloon the II Aumawau free in Meorish Crastal Marlis Richised Terats Callity introundended andfier frehedenoused in 1962 is confe on 

In [ ]:
prompt = "James Bond"
context = torch.tensor(encode(prompt), dtype=torch.long, device=device).view(1, -1)
print(decode(m.generate(context, max_new_tokens=200)[0].tolist()))

James Bondoodernsman for County for Hungta Bedemberal a name the city.
Querlia and the cannike.
The sown mount for the served as actnian Malid and Billade Norali Unjay Campaner.
In 0 16 was He efferers operated


In [ ]:
# Save the model
torch.save(m.state_dict(), 'bigram_language_model.pth')

In [ ]:
# Load the model
loaded_model = BigramLanguageModel()  # Initialize an instance of your model
loaded_model.load_state_dict(torch.load('bigram_language_model.pth'))
loaded_model.to(device).eval()  # Set the model to evaluation mode


BigramLanguageModel(
  (token_embedding_table): Embedding(67, 64)
  (position_embedding_table): Embedding(32, 64)
  (blocks): Sequential(
    (0): Block(
      (sa): MultiHeadAttention(
        (heads): ModuleList(
          (0-3): 4 x Head(
            (key): Linear(in_features=64, out_features=16, bias=False)
            (query): Linear(in_features=64, out_features=16, bias=False)
            (value): Linear(in_features=64, out_features=16, bias=False)
            (dropout): Dropout(p=0.0, inplace=False)
          )
        )
        (proj): Linear(in_features=64, out_features=64, bias=True)
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (ffwd): FeedFoward(
        (net): Sequential(
          (0): Linear(in_features=64, out_features=256, bias=True)
          (1): ReLU()
          (2): Linear(in_features=256, out_features=64, bias=True)
          (3): Dropout(p=0.0, inplace=False)
        )
      )
      (ln1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    

In [ ]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(loaded_model.generate(context, max_new_tokens=2000)[0].tolist()))


Garrually in Blimsling the Aking Cpiculand.
The was mugan and medisor artimes mell that steatly to are bindiling in the functbornst to strumber in firsts now teather in 203 grone for prespons.
It is mulate relequePent at epajyshtengers nontorial releatistan as the frecult Coable Was Roug's.
Pretusion searies and evepress and inin the County 2016 to and in the coveling diven at the gave the AentARF10 player in trist of pWC in the Cour Chumiry whyretainment chus on electronic primm townspolits and year muning perfillied in rightone angemounted as at the S. Amerimel malle La.3 authinworklis publitic freek in the soldided a stable to a ferific.
Heranchoy to peaces had other collong came.
Farrand new was networld gensign mader.
Pladenama is album the teld an mages fru10 177 the after thrank Buleld on Hen lendlus picce series are scine edalled march one by ter.
The 553 highin featured for Vong Kauan Carrosmacila its single deams in admonam.
Atalling parliniqued recomed his electribed to pec